In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  # 화면 상태 체크

import requests
from bs4 import BeautifulSoup
import time  
import os
import random
import csv

In [32]:
url = "https://flight.naver.com/"
browser = webdriver.Chrome()  # 창열기
browser.maximize_window()  # 최대창으로 확대
# 1. naver 페이지 열기
browser.get(url)

# 다른 탭 선택 - 네이버항공권 탭 선택됨.
# browser.switch_to.window(browser.window_handles[1])

In [33]:
# 광고알림창 닫기 버튼
elem = browser.find_element(By.XPATH,'//*[@id="layer"]/button[2]')
elem.click()

In [ ]:
# 1. 김포, 제주 선택
# 출발 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[1]').click()
time.sleep(1)

# 김포 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div/div/ul[1]/li[3]/button').click()
time.sleep(1)

# 도착 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[2]').click()
time.sleep(1)

# 제주 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[2]/div[2]/ul[1]/li[1]/button').click()
time.sleep(1)

In [ ]:
# 1. 네이버항공권 페이지 열기

# 2. 날짜선택 - 시작,끝 날짜

# 3. 검색버튼 클릭

# 4. 스크롤을 하단으로 내리는데 페이지창이 더이상 늘어나지 않을때까지 스크롤 내려야 함.

# 5. 웹스크래핑으로 출력

In [35]:
# 2. 날짜선택 - 시작,끝 날짜
# 달력 열기
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[2]/button[1]')
time.sleep(1)
elem.click()

# 가는날선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[5]/td[2]/button')
time.sleep(1)
elem.click()

# 오는날선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[5]/td[3]/button')
time.sleep(1)
elem.click()

In [36]:
# 검색선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/button')
time.sleep(1)
elem.click()

In [37]:
# 화면이 로딩될때까지 대기
time.sleep(8)
# WebDriverWait(browser,10).until(EC.presence_of_element_located(By.XPATH,'//*[@id="__next"]/div/main/div[4]/div/div[2]/div[2]/div[1]'))

In [38]:
# 화면 스크롤 내림
pre_height = browser.execute_script("return document.body.scrollHeight")  # 현재 화면 높이
pre_height 

6415

In [ ]:
while True:
    # 스크롤 아래로 내림
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    # 추가데이터를 불러오는 동안 대기 (스크롤 내려서 추가 데이터 나오는데 걸리는 시간)
    time.sleep(3)
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if curr_height == pre_height:   # 현재 높이 = 이전 높이  (같으면)
        break
    # 같지 않으면
    pre_height = curr_height
    print("높이 변화 : ",curr_height)

print("스크롤 완료 : ",curr_height)   # 최종 높이


높이 변화 :  15228
높이 변화 :  24053
높이 변화 :  32918
높이 변화 :  33708
스크롤 완료 :  33708


In [ ]:
soup = BeautifulSoup(browser.page_source,"lxml")
results = soup.find("div",{"class":"domestic_results__gp5WB"})  # div의 class가 domestic_results__gp5WB인것 찾음
flights = results.find_all("div",{"class":"domestic_Flight__8bR_b"})
flights[0]  # 첫번째 항공권 출력

<div class="domestic_Flight__8bR_b"><div class="domestic_inner__8geIy"><div class="domestic_schedule__3uLrb"><div class="domestic_item__5CAye"><div class="domestic_heading__eXCgv"><div role="row"><div class="airline_Airline__uUBCa"><span class="airline_logos__Nv1aD" data-area="logos"><img alt="파라타항공" class="airline_logo__S_bQ7" src="https://techminds.pstatic.net/flight/vertical-static/flight/airlines/WE.png?v=3"/></span><span class="airline_text__WWkbY" data-area="texts"><b class="airline_name__0Tw5w">파라타항공</b><span class="airline_info__vBRtW"></span></span></div><hr/></div></div><div class="route_Route__HYsDn"><span class="route_airport__tBD9o"><b class="route_time__xWu7a">06:05</b><i class="route_code__S07WE" data-different="false">GMP</i></span><span class="route_airport__tBD9o"><b class="route_time__xWu7a">07:20</b><i class="route_code__S07WE" data-different="false">CJU</i></span><div class="route_info__RVh13">01시간 15분</div></div></div></div><button class="domestic_select_schedule_

In [ ]:
# < csv파일로 저장 >  csv:하나의 db
# 항공사명,출발시간,도착시간,가격
# 파라타항공,06:05,07:20,26700
f = open("flight.csv","w",encoding="utf-8-sig",newline="")    #  ,newline="" : 줄바꿈 제거
writer = csv.writer(f)
title = ['항공사명','출발시간','도착시간','가격']
writer.writerow(title)   # writer.writerow()를 해야 csv파일로 저장됨

19

In [ ]:
# 항공사명,출발시간,도착시간,가격 출력
for i,flight in enumerate(flights):
    print(f"{i+1}.")
    # 항공사명
    n1 = flight.find("b",{"class":"airline_name__0Tw5w"}).text.strip()  # 변수로 갖고오기
    print(n1,end="\t")
    
    # 출발시간/도착시간
    n2 = flight.find_all("b",{"class":"route_time__xWu7a"})[0].text.strip()
    print(n2,end="\t")  # 출발
    n3 = flight.find_all("b",{"class":"route_time__xWu7a"})[1].text.strip()
    print(n3,end="\t")  # 도착

    # 가격 (타입 - int타입)
    n4 = int(flight.find("i",{"class":"domestic_num__ShOub"}).text.strip().replace(",",""))  #  .replace(",","") : 천단위표시(,) 제거
    print(n4)
    arr2 = [n1,n2,n3,n4]
    writer.writerow(arr2) 
    print("-"*50)

# 파일 닫기  -> 파일을 닫으면 저장됨
# 파일 저장이유 : 정보가져와서 사용
f.close()

1.
파라타항공	06:05	07:20	26700
--------------------------------------------------
2.
진에어	06:05	07:20	31600
--------------------------------------------------
3.
진에어	06:05	07:20	33600
--------------------------------------------------
4.
아시아나항공	06:05	07:20	34700
--------------------------------------------------
5.
파라타항공	06:05	07:20	37700
--------------------------------------------------
6.
대한항공	06:05	07:20	51700
--------------------------------------------------
7.
아시아나항공	06:05	07:20	56700
--------------------------------------------------
8.
파라타항공	06:05	07:20	80700
--------------------------------------------------
9.
진에어	06:05	07:20	81600
--------------------------------------------------
10.
아시아나항공	06:05	07:20	100700
--------------------------------------------------
11.
대한항공	06:05	07:20	105700
--------------------------------------------------
12.
아시아나항공	06:05	07:20	137700
--------------------------------------------------
13.
아시아나항공	06:10	07:25	31700
---------------------------------

In [ ]:
# 항공사명
flights[0].find("b",{"class":"airline_name__0Tw5w"}).text.strip()   

# 출발시간/도착시간
flights[0].find_all("b",{"class":"route_time__xWu7a"})[0].text.strip()  # 출발
flights[0].find_all("b",{"class":"route_time__xWu7a"})[1].text.strip()  # 도착

# 가격 (타입 - int타입)
int(flights[0].find("i",{"class":"domestic_num__ShOub"}).text.strip().replace(",",""))  #  .replace(",","") : 천단위표시(,) 제거

26700

In [ ]:
# 가격
int(flights[0].find("i",{"class":"domestic_num__ShOub"}).text.strip().replace(",",""))  #  .replace(",","") : 천단위표시(,) 제거

26700